# API CALL links
## Proyectos:
"https://reconnectingasia.csis.org/api/projects/?format=json"

## Iniciativas:
https://reconnectingasia.csis.org/api/initiatives/?format=json

## Organizaciones:
https://reconnectingasia.csis.org/api/organizations/?format=json


## Datos geograficos
Vienen distribuidos segun tengan una linea, poligono o cualquier otra figura geometrica
El identifier es el id del proyecto en la db
https://reconnectingasia.csis.org/api/geostore/{identifier}.json?format=json
Aca un ejemplo
https://reconnectingasia.csis.org/api/geostore/be74ab4c-8342-4b2e-ac95-1d2bcca784f2.json?format=json

Pero aparentemente es mejor usar esta otra base de datos donde estan todos los centroides en una sola llamada:
"https://reconnectingasia.csis.org/api/geostore-centroids/?format=json"


# To Webscrape
This data was unavailable in the dbs, so I need to webscrape them:

- Status [in page]
- Description text [in page] (algunas tienen imágenes y links) 
ORGANIZATIONS TYPE
- Contractors [in page]
- Consultants [in page]
- Implementers [in page]
- Operators [in page]

Note> I may need to change user agent, use timesleep, use proxy serves (VPNs), clear cookies, and be careful with honey traps links
https://www.octoparse.com/blog/scrape-websites-without-being-blocked

In [1]:
import requests 
import json
import pandas as pd

#This other API is related to the last one by an ID and has additional info
#The main problem is that it is segmented in groups of 75 features by link 
main_data = "https://reconnectingasia.csis.org/api/projects/?format=json"

In [2]:
response = requests.get(main_data)
#json format
response = response.json()

In [3]:
#get rid of
data = response["results"]

In [47]:
#### Here we create the lists to fill ######
db_id = []
geo_id = [] #In order to match data with the geostore db
name = [] 
planed_completition_date = [] #need to get the day, month and year, from different entrance and save it in text format because some are incomplete
start_date = [] # same as completition  (aparently the start of project)
commencement_date = [] #Aparently the start of works
total_cost = []
total_cost_currency = []

initiatives = [] #Note: some have 2 and others don't have (I should make a list comprehension with and error handle)
funding_sources = [] #Same as initiatives, as there is a db with ORGANIZATIONS and initiatives I may create a 2NF db to save them?
funding_sources_slug = []
funding_amount = []  # Same
funding_currency = [] # Same
project_page_url = []

In [48]:
for x in range(len(data)):
    db_id.append(data[x]["identifier"])
    geo_id.append(data[x]["geo"])
    name.append(data[x]["name"]) 
    planed_completition_date.append([data[x]["planned_completion_day"],data[x]["planned_completion_month"],data[x]["planned_completion_year"]])
    commencement_date.append([data[x]["commencement_day"], data[x]["commencement_month"], data[x]["commencement_year"]])
    start_date.append([data[x]["start_day"], data[x]["start_month"], data[x]["start_year"]])
   
    total_cost.append(data[x]["total_cost"])
    total_cost_currency.append(data[x]["total_cost_currency"])

    initiatives.append([y["name"] for y in data[x]["initiatives"]])
    funding_sources.append([y["sources"][0]["name"] for y in data[x]["funding"] if y["sources"]])
    funding_sources_slug.append([y["sources"][0]["slug"] for y in data[x]["funding"] if y["sources"]])
    funding_amount.append([y["amount"] for y in data[x]["funding"]])
    funding_currency.append([y["currency"] for y in data[x]["funding"]])
    project_page_url.append(data[x]["page_url"])

In [49]:
data_base = {
         "id" : db_id,
        "geo" : geo_id,
        "project" : name,
        "planed_completition" : planed_completition_date,
        "start" : start_date,
        "comencement" : commencement_date,
        "cost" : total_cost,
        "cost_currency" : total_cost_currency,
        "initiatives" : initiatives,
        "funders" : funding_sources,
        "funders_slug" : funding_sources_slug,
        "fund" : funding_amount,
        "fund_currency" : funding_currency,
        "page" : project_page_url,
}

In [51]:
panda = pd.DataFrame(data_base)
###Queda pendiente ver como impedir que los elemntos queden dentro de corchetes

In [52]:
panda.head()

,id,geo,project,planed_completition,start,comencement,cost,cost_currency,initiatives,funders,funders_slug,fund,fund_currency,page
0,3b92029a-1f75-402a-adf5-8d9cf7b3786e,None,1520mm Broad Gauge-Connection Kosice-Vienna,"[None, None, 2033]","[None, None, None]","[None, None, None]",6.490000e+09,EUR,[Belt and Road],[Slovakia],[government-slovakia],[None],[USD],/database/projects/kosice-vienna-railway-const...
1,0b0dd2d6-8b39-44d8-8057-58c88bd74dfd,None,300MW Imported Coal Based Power Project At Gwadar,"[None, None, None]","[None, None, 2017]","[None, None, None]",5.423600e+08,USD,"[Belt and Road, China-Pakistan Economic Corrid...",[],[],[],[],/database/projects/gwadar-coal-fired-power-pla...
2,01b53e04-dd3f-40f3-9e35-2faca340255a,be74ab4c-8342-4b2e-ac95-1d2bcca784f2,4/6 Laning of Ghaziabad-Aligarh section of NH91,"[None, 8, 2017]","[20, 5, 2010]","[None, None, None]",1.141000e+10,INR,[],[Ghaziabad Aligarh Expressway Pvt. Ltd.],[ghaziabad-aligarh-expressway-pvt-ltd],[516700000],[USD],/database/projects/ghaziabad-aligarh-laning-co...
3,21aacd5c-b12b-4048-9e69-343a57c63ee2,23d82ba3-0183-46f3-8536-b71ed70aa43a,4/6-Laning of Solapur-Bijapur section of NH-13...,"[None, None, None]","[None, None, 2012]","[None, None, None]",3.840000e+08,USD,[],"[IndusInd Bank Limited, Bank of India]","[indusind-bank-limited, bank-india]","[74730000, 74730000]","[USD, USD]",/database/projects/solapur-bijapur-road-constr...
4,d86f26cd-178d-4dbd-a60e-562852c617fb,None,4 Laning of Amravati-Jalgaon (PPP),"[None, None, None]","[None, None, 2012]","[None, None, None]",9.189000e+08,USD,[],[ICICI Bank Limited],[icici-bank-limited],[459800000],[USD],/database/projects/amravati-jalgaon-road-toll-...


# Geodata call

In [ ]:
#This API storage all the centroids of the projects
#Centroids are just points, if we want to get the lines from a railroad we need to call api-geostore/identifier
geodata = "https://reconnectingasia.csis.org/api/geostore-centroids/?format=json"

In [2]:
#Make the call the basic geodata first
response = requests.get(all_geodata)
#Save the call in a variable
response_json = response.json()
#Save the straight info
data = response_json["features"]

In [3]:
#Create a series of lists to save the info
geostore_id = []
project_name = []
locations = []
infraestructure_type = []
coordinates = []

#Use a for loop to add the data to the lists
for x in range(len(data)):
    geoestore_id.append(data[x]["id"])
    project_name.append(data[x]['properties']['label'])
    locations.append(data[x]['properties']['locations'])
    infraestructure_type.append(data[x]['properties']['infrastructureType'])
    cost.append(data[x]['properties']['total_cost'])
    currency.append(data[x]['properties']['currency'])
    coordinates.append(data[x]['geometry']['coordinates'])

In [4]:
#Create a dictionary where to put toghether the data
db = {
    "id" : geostore_id,
    "Project" : project_name,
    "Locations" : locations,
    "Infraestructure_type" : infraestructure_type,
    "Cost" : cost,
    "Currency" : currency,
    "Coordinates" : coordinates
}

In [5]:
data_base = pd.DataFrame(db)
#Show the complet float numbers not in cientific notation
pd.options.display.float_format = '{:,.0f}'.format